In [50]:
import praw
from _keys import client_id, secret_key, username, user_agent, password, db_user, db_password, db_name, db_host, db_port
import psycopg2
import time

In [51]:
reddit = praw.Reddit(client_id=client_id, client_secret=secret_key, username=username, password=password, user_agent=user_agent)
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
cur = conn.cursor()

In [52]:
sql = """select sus_user_name
from public.sus_users"""

In [53]:
names = []
cur.execute(sql)
for row in cur:
    names.append(row[0])

In [54]:
sql_insert = """INSERT INTO public.sus_user_comments (author, comment_body, comment_id, comment_score, created_utc, permalink, link_id, edited,
                gilded, distinguished, archived, locked, stickied, controversiality, subreddit_id, subreddit) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [55]:
for name in names:
    try:
        user = reddit.redditor(name)
        comments = user.comments.top(limit=None)
        for comment in comments:
            cur.execute(sql_insert, (name, comment.body, comment.id, comment.score, comment.created, comment.permalink, comment.link_id.split('_')[1], comment.edited, comment.gilded, comment.distinguished, comment.archived, comment.locked, comment.stickied, comment.controversiality, comment.subreddit_id, comment.subreddit.display_name))
            conn.commit()
    except Exception as e:
        print(e)
        continue